In [28]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import time
import pickle

In [29]:
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

In [30]:
train_images, test_images, train_labels, test_labels = train_test_split(X, y, test_size=0.2)

In [31]:
CLASS_NAMES = ["A", "W", "D"]

In [32]:
validation_images, validation_labels = train_images[:5000], train_labels[:5000]
train_images, train_labels = train_images[5000:], train_labels[5000:]

In [33]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

In [18]:
test_ds

<TensorSliceDataset shapes: ((100, 100, 1), ()), types: (tf.uint8, tf.int32)>

In [34]:
def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (100,100))
    return image, label

In [35]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

Training data size: 30887
Test data size: 8972
Validation data size: 5000


In [36]:
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

In [37]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(100,100,1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(3, activation='softmax')
])

In [38]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 23, 23, 96)        11712     
_________________________________________________________________
batch_normalization_5 (Batch (None, 23, 23, 96)        384       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 11, 11, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 256)       614656    
_________________________________________________________________
batch_normalization_6 (Batch (None, 11, 11, 256)       1024      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 256)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 5, 5, 384)        

In [39]:
model.fit(train_ds,
          epochs=20,
          validation_data=validation_ds,
          validation_freq=1)

Epoch 1/20
965/965 [==============================] - 52s 54ms/step - loss: 0.9901 - accuracy: 0.5820 - val_loss: 0.5422 - val_accuracy: 0.7905
Epoch 2/20
965/965 [==============================] - 55s 57ms/step - loss: 0.5464 - accuracy: 0.7802 - val_loss: 0.4130 - val_accuracy: 0.8419
Epoch 3/20
965/965 [==============================] - 55s 57ms/step - loss: 0.3929 - accuracy: 0.8475 - val_loss: 0.3026 - val_accuracy: 0.8860
Epoch 4/20
965/965 [==============================] - 56s 58ms/step - loss: 0.2979 - accuracy: 0.8851 - val_loss: 0.2912 - val_accuracy: 0.8848
Epoch 5/20
965/965 [==============================] - 53s 55ms/step - loss: 0.2289 - accuracy: 0.9122 - val_loss: 0.2462 - val_accuracy: 0.9067
Epoch 6/20
965/965 [==============================] - 54s 56ms/step - loss: 0.1787 - accuracy: 0.9332 - val_loss: 0.2231 - val_accuracy: 0.9141
Epoch 7/20
965/965 [==============================] - 55s 57ms/step - loss: 0.1402 - accuracy: 0.9479 - val_loss: 0.2228 - val_accuracy:

In [40]:
model.evaluate(test_ds)

280/280 [==============================] - 5s 18ms/step - loss: 0.1863 - accuracy: 0.9515


[0.18631482124328613, 0.9514508843421936]

In [41]:
model.save("model.h5")